In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from math import sin, cos
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

np.random.seed(42)

In [4]:
df_6pts = pd.read_csv('../cleaned_data_6pts.csv', index_col=0)
df_6pts.head()

,x2,x3,x4,x5,x6,y2,y3,y4,y5,y6,pitch,yaw,roll
image00002,-0.117864,-0.660870,0.811568,-0.471326,0.388258,0.993030,-0.868580,-0.734130,0.247089,0.431947,-22.874239,1.044306,4.908885
image00004,0.244669,0.390942,0.818068,0.240005,0.475098,0.969607,-0.268480,-0.445271,0.578156,0.481578,26.932741,68.155235,17.243670
image00006,0.475943,0.029326,0.663393,0.186035,0.639567,0.879476,-0.389183,-0.480418,0.351012,0.307038,-10.579652,50.485409,-13.570644
image00008,0.545762,-0.534558,0.548172,-0.038727,0.562912,0.837940,-0.411924,-0.792250,0.351042,0.173244,-10.048455,17.143373,-21.392780
image00010,0.717417,0.251543,0.403701,0.469008,0.526977,0.696644,-0.364394,-0.518721,0.279280,0.265331,-50.544579,68.640549,-59.207973


In [5]:
df_6pts.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2000 entries, image00002 to image04375
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x2      2000 non-null   float64
 1   x3      2000 non-null   float64
 2   x4      2000 non-null   float64
 3   x5      2000 non-null   float64
 4   x6      2000 non-null   float64
 5   y2      2000 non-null   float64
 6   y3      2000 non-null   float64
 7   y4      2000 non-null   float64
 8   y5      2000 non-null   float64
 9   y6      2000 non-null   float64
 10  pitch   2000 non-null   float64
 11  yaw     2000 non-null   float64
 12  roll    2000 non-null   float64
dtypes: float64(13)
memory usage: 218.8+ KB


In [6]:
df_21pts = pd.read_csv('../cleaned_data_21pts.csv', index_col=0)
df_21pts.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,y13,y14,y15,y16,y17,y18,y19,pitch,yaw,roll
image00002,-0.850815,-0.525609,-0.207266,0.387123,0.729734,1.045237,-0.660870,-0.250516,0.378241,0.811568,...,0.028274,0.247089,0.516972,0.431947,0.993030,-0.595580,-0.426537,-22.874239,1.044306,4.908885
image00004,0.418979,0.261841,0.289089,0.480327,0.684867,0.992539,0.390942,0.426828,0.629349,0.818068,...,0.159100,0.578156,0.446953,0.481578,0.969607,0.522405,0.272040,26.932741,68.155235,17.243670
image00006,-0.028031,-0.083471,-0.014014,0.283632,0.487453,0.808384,0.029326,0.155509,0.435379,0.663393,...,0.052335,0.351012,0.349787,0.307038,0.879476,0.037008,-0.114270,-10.579652,50.485409,-13.570644
image00008,-0.726478,-0.638846,-0.417195,0.070929,0.318399,0.667258,-0.534558,-0.258377,0.208870,0.548172,...,-0.006753,0.351042,0.387353,0.173244,0.837940,0.132924,-0.411059,-10.048455,17.143373,-21.392780
image00010,0.252695,0.051993,0.029330,0.103541,0.186478,0.479988,0.251543,0.251446,0.335959,0.403701,...,-0.022183,0.279280,0.287105,0.265331,0.696644,-0.026336,-0.307893,-50.544579,68.640549,-59.207973


In [7]:
df_21pts.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2000 entries, image00002 to image04375
Data columns (total 39 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x1      2000 non-null   float64
 1   x2      2000 non-null   float64
 2   x3      2000 non-null   float64
 3   x4      2000 non-null   float64
 4   x5      2000 non-null   float64
 5   x6      2000 non-null   float64
 6   x7      2000 non-null   float64
 7   x8      2000 non-null   float64
 8   x9      2000 non-null   float64
 9   x10     2000 non-null   float64
 10  x11     2000 non-null   float64
 11  x13     2000 non-null   float64
 12  x14     2000 non-null   float64
 13  x15     2000 non-null   float64
 14  x16     2000 non-null   float64
 15  x17     2000 non-null   float64
 16  x18     2000 non-null   float64
 17  x19     2000 non-null   float64
 18  y1      2000 non-null   float64
 19  y2      2000 non-null   float64
 20  y3      2000 non-null   float64
 21  y4      2000 non-null   flo

In [8]:
# some util functions
def split_x_y(data):
    return data[:, :-3], data[:, -3:]

def split_train_dev_x_y(df, train_size=0.9):
    data_train, data_test = train_test_split(df, train_size=train_size)
    X_train, Y_train = split_x_y(data_train.values)
    X_dev, Y_dev = split_x_y(data_test.values)
    return X_train, Y_train, X_dev, Y_dev 

def draw_axis(img, yaw, pitch, roll, tdx=None, tdy=None, size = 100):

    if tdx != None and tdy != None:
        tdx = tdx
        tdy = tdy
    else:
        height, width = img.shape[:2]
        tdx = width / 2
        tdy = height / 2

    # X-Axis pointing to right. drawn in red
    x1 = size * (cos(yaw) * cos(roll)) + tdx
    y1 = size * (cos(pitch) * sin(roll) + cos(roll) * sin(pitch) * sin(yaw)) + tdy

    # Y-Axis | drawn in green
    #        v
    x2 = size * (-cos(yaw) * sin(roll)) + tdx
    y2 = size * (cos(pitch) * cos(roll) - sin(pitch) * sin(yaw) * sin(roll)) + tdy

    # Z-Axis (out of the screen) drawn in blue
    x3 = size * (sin(yaw)) + tdx
    y3 = size * (-cos(yaw) * sin(pitch)) + tdy

    cv2.line(img, (int(tdx), int(tdy)), (int(x1),int(y1)),(0,0,255),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x2),int(y2)),(0,255,0),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x3),int(y3)),(255,0,0),2)

    return img
    
def show_image(path):
    capture = cv2.VideoCapture(path)
    ret, frame = capture.read()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # convert to RGB format for matplotlib
    landmarks, pitch, yaw, roll = read_mat(path)
    x,y = landmarks[0,32], landmarks[1,32]
    plt.imshow(draw_axis(frame,-yaw, pitch, roll, x, y))
    plt.show()
    capture.release()
    cv2.destroyAllWindows()
    
def try_model(X_train, Y_train, X_dev, Y_dev, model):
    x_model = model.fit(X_train, Y_train)

    print('\nAccuracy on training set: ')
    h = x_model.predict(X_train)
    print('model r2_score =', r2_score(Y_train , h))

    print('\nAccuracy on dev set: ')
    h = x_model.predict(X_dev)
    print('model r2_score =', r2_score(Y_dev, h))
    


def try_models(X_train, Y_train, X_dev, Y_dev, model):
    pitch_model = model().fit(X_train, Y_train[:, 0])
    yaw_model = model().fit(X_train, Y_train[:, 1])
    roll_model = model().fit(X_train, Y_train[:, 2])

    print('\nAccuracy on training set: ')
    h_pitch = pitch_model.predict(X_train)
    h_yaw = yaw_model.predict(X_train)
    h_roll = roll_model.predict(X_train)
    print('Pitch r2_score =', r2_score(Y_train[:, 0] , h_pitch))
    print('Yaw r2_score =', r2_score(Y_train[:, 1] , h_yaw))
    print('Roll r2_score =', r2_score(Y_train[:, 2] , h_roll))

    print('\nAccuracy on dev set: ')
    h_pitch = pitch_model.predict(X_dev)
    h_yaw = yaw_model.predict(X_dev)
    h_roll = roll_model.predict(X_dev)
    print('Pitch r2_score =', r2_score(Y_dev[:, 0] , h_pitch))
    print('Yaw r2_score =', r2_score(Y_dev[:, 1] , h_yaw))
    print('Roll r2_score =', r2_score(Y_dev[:, 2] , h_roll))


In [9]:
from sklearn.linear_model import ElasticNet, LinearRegression

# Try LinearRegression on 6pts df.
X_train, Y_train, X_dev, Y_dev = split_train_dev_x_y(df_6pts)
try_models(X_train, Y_train, X_dev, Y_dev, LinearRegression)


Accuracy on training set: 
Pitch r2_score = 0.28351379671854116
Yaw r2_score = 0.9423150777785204
Roll r2_score = 0.32406461198752445

Accuracy on dev set: 
Pitch r2_score = -0.1876087648539888
Yaw r2_score = 0.6141699816215083
Roll r2_score = 0.1437779712699443


In [10]:
# Try LinearRegression on 21pts df.
X_train, Y_train, X_dev, Y_dev = split_train_dev_x_y(df_21pts)
try_models(X_train, Y_train, X_dev, Y_dev, LinearRegression)


Accuracy on training set: 
Pitch r2_score = 0.26161394388920145
Yaw r2_score = 0.9128821726824775
Roll r2_score = 0.33351167114593616

Accuracy on dev set: 
Pitch r2_score = 0.25823928001270646
Yaw r2_score = 0.9737078246591422
Roll r2_score = 0.1714958862736281


In [11]:
# Try LinearRegression on 68pts df.
df_68pts = pd.read_csv('../cleaned_data_68pts.csv', index_col=0)
X_train, Y_train, X_dev, Y_dev = split_train_dev_x_y(df_68pts)
try_models(X_train, Y_train, X_dev, Y_dev, LinearRegression)


Accuracy on training set: 
Pitch r2_score = 0.4353997129085946
Yaw r2_score = 0.9415563839272578
Roll r2_score = 0.5450867949130696

Accuracy on dev set: 
Pitch r2_score = 0.15081998673095276
Yaw r2_score = 0.9726124666769752
Roll r2_score = 0.06459301593864275


In [12]:
# Try ElasticNet on 6pts df.
X_train, Y_train, X_dev, Y_dev = split_train_dev_x_y(df_6pts)
try_models(X_train, Y_train, X_dev, Y_dev, ElasticNet)


Accuracy on training set: 
Pitch r2_score = 0.0267533628935116
Yaw r2_score = 0.5143595335090418
Roll r2_score = 0.04772667209229786

Accuracy on dev set: 
Pitch r2_score = 0.08699483606238356
Yaw r2_score = 0.5667318791119629
Roll r2_score = 0.21098183183364227


In [71]:
# Try ElasticNet on 21pts df.
X_train, Y_train, X_dev, Y_dev = split_train_dev_x_y(df_21pts)
try_models(X_train, Y_train, X_dev, Y_dev, ElasticNet)


Accuracy on training set: 
Pitch r2_score = 0.10235564902711303
Yaw r2_score = 0.8322152580508285
Roll r2_score = 0.08396555470645117

Accuracy on dev set: 
Pitch r2_score = 0.2956113212265302
Yaw r2_score = 0.8837774039045806
Roll r2_score = 0.1934995962296875


In [74]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(2)
Xt = poly.fit_transform(X_train)
Xd = poly.fit_transform(X_dev)

try_model(Xt, Y_train[:, 0], Xd, Y_dev[:, 0], model=Lasso())


Accuracy on training set: 
model r2_score = 0.18824938320770335

Accuracy on dev set: 
model r2_score = 0.503883569077211


In [16]:
from sklearn.ensemble import RandomForestRegressor
try_model(X_train, Y_train[:, 1], X_dev, Y_dev[:, 1], model=RandomForestRegressor())
# try_models(X_train, Y_train, X_dev, Y_dev, RandomForestRegressor)


Accuracy on training set: 
model r2_score = 0.9821475147013521

Accuracy on dev set: 
model r2_score = 0.9983614809557879


In [17]:
from sklearn.svm import SVR
try_model(X_train, Y_train[:, 0], X_dev, Y_dev[:, 0], model=RandomForestRegressor())
# model_yaw = svm.SVR(C=15,epsilon=0.5).fit(X_train_yaw, y_train_yaw)


Accuracy on training set: 
model r2_score = 0.7876306585701807

Accuracy on dev set: 
model r2_score = 0.8245198477765274


In [27]:

try_model(X_train, Y_train[:, 0], X_dev, Y_dev[:, 0], model=SVR(kernel='rbf', C=500, gamma=0.1, epsilon=.1))


Accuracy on training set: 
model r2_score = 0.42296884396713197

Accuracy on dev set: 
model r2_score = 0.805534780113009


In [96]:
from xgboost import XGBRegressor

try_model(X_train, Y_train[:, 2], X_dev, Y_dev[:, 2],
model=XGBRegressor(n_estimators=2048, learning_rate=0.005, min_child_weight=110, max_leaves=10, subsample=0.8, colsample_bytree=1, nthread=4))


Accuracy on training set: 
model r2_score = 0.39560598641995437

Accuracy on dev set: 
model r2_score = 0.5629683047813534


In [31]:
try_model(X_train, Y_train[:, 2], X_dev, Y_dev[:, 2], model=SVR(kernel='rbf', C=1000, gamma=0.1, epsilon=.1))


Accuracy on training set: 
model r2_score = 0.8483386284879297

Accuracy on dev set: 
model r2_score = 0.8349226456330547


In [43]:
try_model(X_train, Y_train[:, 2], X_dev, Y_dev[:, 2], model=RandomForestRegressor(n_estimators=500, max_depth=12))


Accuracy on training set: 
model r2_score = 0.9528649838543892

Accuracy on dev set: 
model r2_score = 0.8096840938911147


In [48]:
try_model(X_train, Y_train[:, 2], X_dev, Y_dev[:, 2], model=SVR(C=1200, gamma=0.1))


Accuracy on training set: 
model r2_score = 0.8533304167601464

Accuracy on dev set: 
model r2_score = 0.8395445859223127


In [49]:
X_train, Y_train, X_dev, Y_dev = split_train_dev_x_y(df_68pts)
try_model(X_train, Y_train[:, 2], X_dev, Y_dev[:, 2], model=SVR(C=500, gamma=0.1))


Accuracy on training set: 
model r2_score = 0.7675359338579909

Accuracy on dev set: 
model r2_score = 0.47552051146542185


In [50]:
try_model(X_train, Y_train[:, 2], X_dev, Y_dev[:, 2], model=SVR(C=1000, gamma=0.1))


Accuracy on training set: 
model r2_score = 0.8650551457825009

Accuracy on dev set: 
model r2_score = 0.7173239792861206


In [52]:
try_model(X_train, Y_train[:, 2], X_dev, Y_dev[:, 2], model=RandomForestRegressor(n_estimators=500))


Accuracy on training set: 
model r2_score = 0.8958606888211691

Accuracy on dev set: 
model r2_score = 0.7806205400979032


In [54]:
try_model(X_train, Y_train[:, 2], X_dev, Y_dev[:, 2], model=XGBRegressor(n_estimators=1000, nthread=4))


Accuracy on training set: 
model r2_score = 0.9999999994731967

Accuracy on dev set: 
model r2_score = 0.9694965257926832


In [61]:
try_model(X_train, Y_train[:, 0], X_dev, Y_dev[:, 0], model=XGBRegressor(n_estimators=50, nthread=4))


Accuracy on training set: 
model r2_score = 0.9993243360245276

Accuracy on dev set: 
model r2_score = -3.392971184240098


In [62]:
try_model(X_train, Y_train[:, 0], X_dev, Y_dev[:, 0], model=LinearRegression())


Accuracy on training set: 
model r2_score = 0.5443937954218017

Accuracy on dev set: 
model r2_score = -0.5309620977576852
